In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
from keras.layers import Dense
from keras.models import Sequential

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

C:\Users\andrew\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Training Data

In [2]:
df= pd.read_csv('train.csv')
df.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26570 non-null  int64  
 1   product_code    26570 non-null  object 
 2   loading         26320 non-null  float64
 3   attribute_0     26570 non-null  object 
 4   attribute_1     26570 non-null  object 
 5   attribute_2     26570 non-null  int64  
 6   attribute_3     26570 non-null  int64  
 7   measurement_0   26570 non-null  int64  
 8   measurement_1   26570 non-null  int64  
 9   measurement_2   26570 non-null  int64  
 10  measurement_3   26189 non-null  float64
 11  measurement_4   26032 non-null  float64
 12  measurement_5   25894 non-null  float64
 13  measurement_6   25774 non-null  float64
 14  measurement_7   25633 non-null  float64
 15  measurement_8   25522 non-null  float64
 16  measurement_9   25343 non-null  float64
 17  measurement_10  25270 non-null 

In [4]:
df.isnull().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [5]:
#solving the nan issue
df.iloc[:,10:-1].columns

Index(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6',
       'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10',
       'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14',
       'measurement_15', 'measurement_16', 'measurement_17'],
      dtype='object')

In [6]:
df.loading.fillna(df.loading.mean(),inplace=True)
for i in df.iloc[:,10:-1].columns:
    df['{}'.format(i)].fillna(df['{}'.format(i)].mean(),inplace=True)

In [7]:
df.isnull().sum()

id                0
product_code      0
loading           0
attribute_0       0
attribute_1       0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
failure           0
dtype: int64

In [8]:
#preparing categorical variables
df_enc = pd.get_dummies(df,drop_first=True)
df_enc.head()

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,...,measurement_16,measurement_17,failure,product_code_B,product_code_C,product_code_D,product_code_E,attribute_0_material_7,attribute_1_material_6,attribute_1_material_8
0,0,80.10,9,5,7,8,4,18.040,12.518,15.748,...,14.684,764.100,0,0,0,0,0,1,0,1
1,1,84.89,9,5,14,3,3,18.213,11.540,17.717,...,15.631,682.057,0,0,0,0,0,1,0,1
2,2,82.43,9,5,12,1,5,18.057,11.652,16.738,...,17.946,663.376,0,0,0,0,0,1,0,1
3,3,101.07,9,5,13,2,6,17.295,11.188,18.576,...,17.172,826.282,0,0,0,0,0,1,0,1
4,4,188.06,9,5,9,2,8,19.346,12.950,16.990,...,16.412,579.885,0,0,0,0,0,1,0,1


In [9]:
#splitting training and testting set
x = df_enc.drop('failure',axis=1)
y = df_enc['failure']

In [10]:
par = 'failure ~'
for i in x.columns:
    par += ' {} +'.format(i)
par=par[:-1]
par

'failure ~ id + loading + attribute_2 + attribute_3 + measurement_0 + measurement_1 + measurement_2 + measurement_3 + measurement_4 + measurement_5 + measurement_6 + measurement_7 + measurement_8 + measurement_9 + measurement_10 + measurement_11 + measurement_12 + measurement_13 + measurement_14 + measurement_15 + measurement_16 + measurement_17 + product_code_B + product_code_C + product_code_D + product_code_E + attribute_0_material_7 + attribute_1_material_6 + attribute_1_material_8 '

In [11]:
# feature selection
stat_model = smf.glm(formula = par,data = df_enc).fit()
stat_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                failure   No. Observations:                26570
Model:                            GLM   Df Residuals:                    26545
Model Family:                Gaussian   Df Model:                           24
Link Function:               identity   Scale:                         0.16439
Method:                          IRLS   Log-Likelihood:                -13703.
Date:                Mon, 27 Feb 2023   Deviance:                       4363.8
Time:                        15:17:27   Pearson chi2:                 4.36e+03
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01910
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0049      0.006     -0.755      0.450      -0.018       0.008
id                      1.888e-07   1.62e-06      0.117      0.907   -2.98e-06    3.36e-06
loading                    0.0014   6.41e-05     21.112      0.000       0.001       0.001
attribute_2               -0.0063      0.010     -0.640      0.522      -0.026       0.013
attribute_3               -0.0134      0.012     -1.122      0.262      -0.037       0.010
measurement_0              0.0003      0.001      0.428      0.669      -0.001       0.002
measurement_1             -0.0003      0.001     -0.379      0.705      -0.002       0.001
measurement_2              0.0026      0.001      3.335      0.001       0.001       0.004
measurement_3              0.0009      0.003      0.355      0.723      -0.004       0.006
measurement_4             -0.0055      0.003     -2.085      0.037      -0.011      -0.000
measurement_5              0.0033      0.003      1.051      0.293      -0.003       0.009
measurement_6              0.0025      0.003      0.888      0.374      -0.003       0.008
measurement_7              0.0036      0.003      1.269      0.204      -0.002       0.009
measurement_8              0.0022      0.003      0.701      0.483      -0.004       0.008
measurement_9             -0.0025      0.003     -0.965      0.335      -0.008       0.003
measurement_10            -0.0006      0.002     -0.349      0.727      -0.004       0.003
measurement_11            -0.0018      0.002     -1.044      0.296      -0.005       0.002
measurement_12             0.0010      0.002      0.538      0.591      -0.003       0.004
measurement_13            -0.0006      0.002     -0.251      0.802      -0.005       0.004
measurement_14             0.0015      0.002      0.836      0.403      -0.002       0.005
measurement_15             0.0003      0.002      0.157      0.875      -0.003       0.004
measurement_16             0.0009      0.002      0.582      0.561      -0.002       0.004
measurement_17          8.703e-05   3.51e-05      2.481      0.013    1.83e-05       0.000
product_code_B             0.0005      0.010      0.048      0.962      -0.018       0.019
product_code_C            -0.0024      0.006     -0.395      0.693      -0.014       0.010
product_code_D            -0.0184      0.022     -0.833      0.405      -0.062       0.025
product_code_E             0.0088      0.008      1.070      0.285      -0.007       0.025
attribute_0_material_7    -0.0054      0.016     -0.338      0.735      -0.036       0.026
attribute_1_material_6     0.0088      0.008      1.070      0.285      -0.007       0.025
attribute_1_material_8     0.0042      0.007      0.614      0.539      -0.009       0.018
================================================================================

In [12]:
pval = pd.DataFrame(stat_model.pvalues)
sig_col = pval[pval < 0.05].dropna().index
sig_col

Index(['loading', 'measurement_2', 'measurement_4', 'measurement_17'], dtype='object')

In [13]:
x_new = df_enc[sig_col]
sc = StandardScaler()

X_train , X_test,y_train,y_test = train_test_split(x_new,y,test_size = 0.1 , random_state=50)

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
model = Sequential(
    [
    Dense(78,activation= 'relu'),
    Dense(156,activation='relu'),
    Dense(78,activation='relu'),
    Dense(1,activation = 'sigmoid')
    ]
)
model.compile(
    loss= 'binary_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)
model.fit(X_train,y_train,epochs=350,use_multiprocessing=True)


Epoch 1/350
748/748 [==============================] - 1s 712us/step - loss: 0.5157 - accuracy: 0.7877
Epoch 2/350
748/748 [==============================] - 1s 710us/step - loss: 0.5095 - accuracy: 0.7884
Epoch 3/350
748/748 [==============================] - 1s 693us/step - loss: 0.5087 - accuracy: 0.7884
Epoch 4/350
748/748 [==============================] - 1s 704us/step - loss: 0.5083 - accuracy: 0.7884
Epoch 5/350
748/748 [==============================] - 1s 698us/step - loss: 0.5078 - accuracy: 0.7884
Epoch 6/350
748/748 [==============================] - 1s 710us/step - loss: 0.5076 - accuracy: 0.7884
Epoch 7/350
748/748 [==============================] - 1s 684us/step - loss: 0.5076 - accuracy: 0.7884
Epoch 8/350
748/748 [==============================] - 1s 730us/step - loss: 0.5072 - accuracy: 0.7884
Epoch 9/350
748/748 [==============================] - 1s 725us/step - loss: 0.5070 - accuracy: 0.7884
Epoch 10/350
748/748 [==============================] - 1s 718us/step - l

748/748 [==============================] - 1s 725us/step - loss: 0.4859 - accuracy: 0.7924
Epoch 80/350
748/748 [==============================] - 1s 711us/step - loss: 0.4851 - accuracy: 0.7927
Epoch 81/350
748/748 [==============================] - 1s 709us/step - loss: 0.4848 - accuracy: 0.7926
Epoch 82/350
748/748 [==============================] - 1s 741us/step - loss: 0.4842 - accuracy: 0.7933
Epoch 83/350
748/748 [==============================] - 1s 707us/step - loss: 0.4832 - accuracy: 0.7923
Epoch 84/350
748/748 [==============================] - 1s 708us/step - loss: 0.4835 - accuracy: 0.7931
Epoch 85/350
748/748 [==============================] - 1s 709us/step - loss: 0.4830 - accuracy: 0.7930
Epoch 86/350
748/748 [==============================] - 1s 709us/step - loss: 0.4818 - accuracy: 0.7941
Epoch 87/350
748/748 [==============================] - 1s 708us/step - loss: 0.4807 - accuracy: 0.7939
Epoch 88/350
748/748 [==============================] - 1s 717us/step - loss:

748/748 [==============================] - 1s 701us/step - loss: 0.4406 - accuracy: 0.8094
Epoch 158/350
748/748 [==============================] - 1s 700us/step - loss: 0.4405 - accuracy: 0.8095
Epoch 159/350
748/748 [==============================] - 1s 700us/step - loss: 0.4400 - accuracy: 0.8097
Epoch 160/350
748/748 [==============================] - 1s 731us/step - loss: 0.4401 - accuracy: 0.8104
Epoch 161/350
748/748 [==============================] - 1s 699us/step - loss: 0.4376 - accuracy: 0.8110
Epoch 162/350
748/748 [==============================] - 1s 706us/step - loss: 0.4389 - accuracy: 0.8111
Epoch 163/350
748/748 [==============================] - 1s 697us/step - loss: 0.4369 - accuracy: 0.8104
Epoch 164/350
748/748 [==============================] - 1s 703us/step - loss: 0.4359 - accuracy: 0.8108
Epoch 165/350
748/748 [==============================] - 1s 702us/step - loss: 0.4351 - accuracy: 0.8128
Epoch 166/350
748/748 [==============================] - 1s 699us/ste

748/748 [==============================] - 1s 699us/step - loss: 0.4081 - accuracy: 0.8228
Epoch 236/350
748/748 [==============================] - 1s 688us/step - loss: 0.4078 - accuracy: 0.8234
Epoch 237/350
748/748 [==============================] - 1s 690us/step - loss: 0.4066 - accuracy: 0.8247
Epoch 238/350
748/748 [==============================] - 1s 722us/step - loss: 0.4064 - accuracy: 0.8252
Epoch 239/350
748/748 [==============================] - 1s 689us/step - loss: 0.4055 - accuracy: 0.8241
Epoch 240/350
748/748 [==============================] - 1s 685us/step - loss: 0.4082 - accuracy: 0.8239
Epoch 241/350
748/748 [==============================] - 1s 702us/step - loss: 0.4059 - accuracy: 0.8263
Epoch 242/350
748/748 [==============================] - 1s 688us/step - loss: 0.4036 - accuracy: 0.8262
Epoch 243/350
748/748 [==============================] - 1s 691us/step - loss: 0.4051 - accuracy: 0.8260
Epoch 244/350
748/748 [==============================] - 1s 702us/ste

748/748 [==============================] - 1s 707us/step - loss: 0.3815 - accuracy: 0.8360
Epoch 314/350
748/748 [==============================] - 1s 714us/step - loss: 0.3826 - accuracy: 0.8352
Epoch 315/350
748/748 [==============================] - 1s 731us/step - loss: 0.3849 - accuracy: 0.8357
Epoch 316/350
748/748 [==============================] - 1s 708us/step - loss: 0.3824 - accuracy: 0.8362
Epoch 317/350
748/748 [==============================] - 1s 755us/step - loss: 0.3800 - accuracy: 0.8368
Epoch 318/350
748/748 [==============================] - 1s 702us/step - loss: 0.3821 - accuracy: 0.8358
Epoch 319/350
748/748 [==============================] - 1s 702us/step - loss: 0.3815 - accuracy: 0.8366
Epoch 320/350
748/748 [==============================] - 1s 718us/step - loss: 0.3803 - accuracy: 0.8374
Epoch 321/350
748/748 [==============================] - 1s 700us/step - loss: 0.3802 - accuracy: 0.8371
Epoch 322/350
748/748 [==============================] - 1s 710us/ste

In [15]:
#from keras.utils import to_categorical
#y_train = to_categorical(y_train)  
#y_test = to_categorical(y_test)  

model = XGBClassifier(n_estimators =1000,max_depth=50)
model.fit(X_train,y_train)

model = RandomForestClassifier()
model.fit(X_train,y_train)

model = DecisionTreeClassifier()
model.fit(X_train,y_train)

In [ ]:
def converted_pred(y):
    y = (y > 0.5 ).astype('int32')
    y=y[:,0]
    return y

In [ ]:
#y_train_pred =model.predict(X_train)

# Preformance ON Training Data

In [ ]:
#f1_score(y_train,y_int)

# Preformance ON Testing Data

In [ ]:
y_pred =model.predict(X_test)
y_pred.max()

In [ ]:
y_pred = converted_pred(y_pred)

In [ ]:
(y_pred == y_test).mean()

In [ ]:
y_test

In [ ]:
#f1_score(y_test,y_pred)

In [ ]:
np.bincount(y_pred)

# Testing Data

In [ ]:
df_test= pd.read_csv('test.csv')
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.loading.fillna(df_test.loading.mean(),inplace=True)
for i in df_test.iloc[:,10::].columns:
    df_test['{}'.format(i)].fillna(df_test['{}'.format(i)].mean(),inplace=True)

In [ ]:
df_test.isnull().sum()

In [ ]:
#preparing categorical variables
df_test_enc = pd.get_dummies(df_test,drop_first=True)
df_test_enc.head()

In [ ]:
test = df_test[sig_col]
test = sc.fit_transform(test)
test.shape

In [ ]:
y_pred_test = converted_pred(model.predict(test))
y_pred_test

In [ ]:
y_pred_test.sum()

In [ ]:
np.bincount(y_pred_test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.failure[0:]=y_pred_test
submission

In [ ]:
np.bincount(submission.failure)

In [ ]:
submission.to_csv('final.csv',index=False)